In [2]:
def min_max(arr):
    comparisons = 0

    if len(arr) % 2 == 0:
        comparisons += 1
        if arr[0] < arr[1]:
            min_val, max_val = arr[0], arr[1]
        else:
            min_val, max_val = arr[1], arr[0]
        start = 2
    else:
        min_val = max_val = arr[0]
        start = 1

    for i in range(start, len(arr), 2):
        comparisons += 1
        if arr[i] < arr[i + 1]:
            small, large = arr[i], arr[i + 1]
        else:
            small, large = arr[i + 1], arr[i]

        comparisons += 1
        if small < min_val:
            min_val = small

        comparisons += 1
        if large > max_val:
            max_val = large

    return min_val, max_val, comparisons

In [3]:
import random

"""
re-used from quicksort
"""
def partition(a, p, r):
    x = a[r]
    i = p - 1
    for j in range(p, r):
        if a[j] <= x:
            i += 1
            a[i], a[j] = a[j], a[i]
    a[i + 1], a[r] = a[r], a[i + 1]
    return i + 1
    
"""
re-used from quicksort
"""
def randomized_partition(a, p, r):
    i = random.randint(p, r)
    a[r], a[i] = a[i], a[r]
    return partition(a, p, r)

In [5]:
"""
i = 0 | smallest
i = len(a) // 2 | median
i = len(a) - 1 | largest
"""
def randomized_select(a, p, r, i):
    if p == r:
        return a[p]
    q = randomized_partition(a, p, r)
    k = q - p
    if i == k:
        return a[q]
    elif i < k:
        return randomized_select(a, p, q - 1, i)
    else:
        return randomized_select(a, q + 1, r, i - k - 1)

In [6]:
"""
iterative version
"""
def randomized_select_iterative(a, i):
    p = 0
    r = len(a) - 1
    while True:
        if p == r:
            return a[p]

        q = randomized_partition(a, p, r)
        k = q - p
        if i == k:
            return a[q]
        elif i < k:
            r = q - 1
        else:
            i = i - k - 1
            p = q + 1

In [7]:
def partition_around(a, p, r, x):
    for i in range(p, r + 1):
        if a[i] == x:
            a[i], a[r] = a[r], a[i]
            break

    pivot = a[r]
    i = p - 1

    for j in range(p, r):
        if a[j] <= pivot:
            i += 1
            a[i], a[j] = a[j], a[i]

    a[i + 1], a[r] = a[r], a[i + 1]
    return i + 1

In [9]:
"""
Pivot is not random
Pivot is guaranteed to be "good enough"

At least:
3n/10 elements are discarded every level

Worst-case O(n)
"""

def select(arr, k):
    if len(arr) == 1:
        return arr[0]

    groups = [arr[i:i+5] for i in range(0, len(arr), 5)]

    medians = []
    for group in groups:
        group.sort()
        medians.append(group[len(group)//2])

    pivot = select(medians, (len(medians) + 1)//2)

    lows = [x for x in arr if x < pivot]
    highs = [x for x in arr if x > pivot]
    pivots = [x for x in arr if x == pivot]

    if k <= len(lows):
        return select(lows, k)
    elif k <= len(lows) + len(pivots):
        return pivot
    else:
        return select(highs, k - len(lows) - len(pivots))

In [11]:
def deterministic_quicksort(a):
    def quicksort_helper(a, p, r):
        if p < r:
            n = r - p + 1
            median = select(a[p:r+1], (n + 1) // 2)  # deterministic SELECT

            q = partition_around(a, p, r, median)

            quicksort_helper(a, p, q - 1)
            quicksort_helper(a, q + 1, r)

    quicksort_helper(a, 0, len(a) - 1)

In [ ]:
+----+-------------------+-------------------------------+-------------------------------------------+
| #  | Comparison        | Resulting knowledge            | Purpose                                   |
+----+-------------------+-------------------------------+-------------------------------------------+
| 1  | compare(a, b)     | a ≤ b  (swap if needed)        | Order first pair                          |
| 2  | compare(c, d)     | c ≤ d  (swap if needed)        | Order second pair                         |
| 3  | compare(a, c)     | a ≤ c  (swap pairs if needed)  | Ensure a is smallest of {a,b,c,d}         |
| 4  | compare(b, d)     | min(b,d) ≤ max(b,d)            | Eliminate largest from median candidates |
| 5  | compare(e, c)     | e ≤ c  or  e > c               | Place e relative to lower half            |
| 6  | compare(e, min(b,d)) | final median determined     | Decide between remaining candidates       |
+----+-------------------+-------------------------------+-------------------------------------------+
"""
# after steps 1–4
candidates = [c, min(b, d), e]

# after step 5
if e <= c:
    median = max(e, a)
else:
    median = min(e, min(b, d))
"""

In [13]:
def quantiles(A, k):
    n = len(A)
    if k <= 1 or n == 0:
        return []

    m = (k + 1) // 2  # ceil(k / 2)

    rank = (m * n + k - 1) // k

    q = select(A, rank)

    L = [x for x in A if x < q]
    R = [x for x in A if x > q]

    return quantiles(L, m) + [q] + quantiles(R, k - m)

In [15]:
def k_closest_to_median(S, k):
    n = len(S)
    
    median = select(S, (n + 1) // 2)
    distances = [abs(x - median) for x in S]
    d_k = select(distances, k)

    return [x for x in S if abs(x - median) <= d_k]

In [17]:
"""
X = [1, 3, 5, 7]
Y = [2, 4, 6, 8]
m1 = 5, m2 = 6

m1 < m2 → discard first half of X ([1,3]) and second half of Y ([6,8])
Remaining: X=[5,7], Y=[2,4]

Recurse: m1=7, m2=4, m1>m2 → discard second half of X ([7]) and first half of Y ([2])
Remaining: X=[5], Y=[4] → median = (max(5,4) + min(...))/2 = 5
"""
def median_two_sorted(X, Y):
    n = len(X)
    if n == 1:
        return (X[0] + Y[0]) / 2
    if n == 2:
        return (max(X[0], Y[0]) + min(X[1], Y[1])) / 2

    m1 = X[n//2]
    m2 = Y[n//2]

    if m1 == m2:
        return m1

    elif m1 < m2:
        return median_two_sorted(X[n//2:], Y[:n - n//2])
    else:
        return median_two_sorted(X[:n - n//2], Y[n//2:])